**Notebook to Test API**


In [65]:
#Imports
from getpass import getpass
import os
import urllib.parse
import requests


In [66]:
#Manually Enter API Key:
os.environ['EVERY_API_KEY'] = getpass('Every.org API key: ')


In [67]:
API_KEY = os.getenv('EVERY_API_KEY', 'XXX').strip()
if API_KEY in ('', 'XXX'):
    raise RuntimeError(
        'Set your Every.org API key in the EVERY_API_KEY environment variable or replace the placeholder before running.')
BASE_URL = "https://partners.every.org/v0.2/search" 
session = requests.Session()
session.headers.update({'Accept': 'application/json'})

In [68]:
#Test Step:
term = "education"
resp = session.get(f"{BASE_URL}/{term}", params={"apiKey": API_KEY, "take": 1})
print(resp.status_code)
print(resp.text) 

200
{"nonprofits":[{"description":"An organization focused on science and technology. It received its nonprofit status in 2020.","ein":"843904403","name":"Panoscience Education","profileUrl":"https://www.every.org/panoscience-education","matchedTerms":["education"],"slug":"panoscience-education","location":"COCOA, FL","hasAdmin":false,"tags":["libraries","scholarships","education"],"donationsEnabled":true}]}


In [69]:
#Simple Search Function
def search_nonprofits(term: str | None = None, *, ntee_code: str | None = None, limit: int = 3, near: str | None = None):
    """Search Every.org nonprofits by text term or NTEE code."""
    if not API_KEY:
        raise RuntimeError('Set your EVERY_API_KEY environment variable first.')
    if not term and not ntee_code:
        raise ValueError('Provide either a search term or an NTEE code.')
    params = {'apiKey': API_KEY, 'take': limit}
    if near:
        params['near'] = near
    if ntee_code:
        params['nteeCode'] = ntee_code
        url = BASE_URL
    else:
        url = f"{BASE_URL}/{urllib.parse.quote(term)}"
    resp = requests.get(url, params=params, timeout=10)
    if not resp.ok:
        raise RuntimeError(
            f"Every.org request failed: {resp.status_code} {resp.reason}\n"
            f"URL: {resp.url}\n"
            f"Response body: {resp.text}"
        )
    return resp.json().get('nonprofits', [])


In [70]:
#Test Search Function
results = search_nonprofits(ntee_code='D20', limit=3)
for n in results:
    ntee = n.get('nteeCodeMeaning', {}) or {}
    print(f"{n['name']}: {ntee.get('decileMeaning', 'NTEE detail unavailable')}")


RuntimeError: Every.org request failed: 404 Not Found
URL: https://partners.every.org/v0.2/search?apiKey=pk_live_eb9141b3fe7b78b7644c2c838016abd5&take=3&nteeCode=D20
Response body: Not found